In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer# Kelîmeleri simgeleştirmek için
from keras.layers import Dense, GRU, Input, Embedding# Sinir ağı katmanları
from tensorflow.keras.models import Model# Fonksiyonel API kullanarak modeli oluşturmak için
from keras.callbacks import ModelCheckpoint# Sinir ağının eğitim sırasında kaydedilmesi için
from tensorflow.keras.preprocessing.sequence import pad_sequences# Cümleleri eşit uzunluğa getirmek için
from keras.losses import SparseCategoricalCrossentropy# Kayıp fonksiyonuyla ilgili detaylar için
from keras.optimizers import RMSprop# RMSProp yinelenen sinir ağlarında daha başarılı
import numpy as nm

### Veri seti ve kullanılan dil modelleri hakkında:
- Veri seti : Çeviri için akla gelen ilk veri seti kaynaklarından birisi tatoeba.org adresidir. Buradaki nisbeten kısa cümleler; fakat şu an mükemmel bir sistem tasarlamaya çalışmadığımız için ve veri büyüklüğü iyi seviyede olduğu için kullanılabilir. Veri seti, https://tatoeba.org/tr/downloads adresinden indirilmiştir. Websitesinin anasayfasının adresi https://tatoeba.org/tr 'dir. Cümleler CC-BY lisansı ve bu lisansın farklı versiyonları altında sunulmaktadır.
- İngilizce kelîme vektörü veri seti : Stanford Üniversitesi'nin hâzırladığı GLoVe kelîme vektör modeli (https://github.com/stanfordnlp/GloVe?tab=readme-ov-file), kamu malı tahsis lisansı (Open Data Commons Public Domain Dedication and License (PDDL)) altında sunulmaktadır.
- Türkçe kelîme vektörü veri seti : MIT lisansıyla sunulan, eğitilmiş bir Word2Vec kelîme vektör modeli (https://github.com/akoksal/Turkish-Word2Vec?tab=readme-ov-file)
- Aslında, veri setini Embedding katmanıyla gömüp, ağırlıkların eğitim süresince değişebilir olmasını sağlamak da kâfî olabilir; fakat daha kaliteli bir model için Embedding katmanlarının ağırlıklarını ilgili dil için hâzırlanmış bir kelîme vektörüyle temsil etmek lazım.

### Proje detayları:
- Uçtan uca (sequence to sequence) bir makine çevirisi ağı oluşturmak istiyoruz.
- Bunun için iki yapı kullanmalıyız: encoder (kodlayıcı) ve decoder (kod çözücü)
- Kodlayıcı, kaynak dildeki cümleyi alır ve bir düşünce matrisi oluşturur
- Kod çözücü bu düşünce matrisini alır ve hedef dile çeviri yapar.
- Eğitimde, kod çözücüye hem düşünce matrisini, hem de hedef çevirinin ilk kelîmesini vermemiz gerekiyor; böylece modelin sonraki kelîme doğru tahmîn etmeyi öğrenmesini bekliyoruz. Elbette bunun için geri yayılım uygulayan sinir ağına ihtiyacımız var.
- Kod çözücü veyâ çözümleyici kaynak cümleyi hedef cümleye çevirirken sonraki kelîmeyi tahmîn ederek devâm etmektedir; çözümleyicinin cümlenin nerede başlayıp, nerede biteceğini tahmîn edebilmesi için cümle başını ve sonunu bir 'token' olarak işâretlememiz gerekiyor. Bunun için metîn içerisinde geçmeyen özel bir karakter dizisi (`<START>` ve `<END>` gibi) seçmeliyiz; fakat biz '<' ve '>' karakterlerini veri setinden çıkarmak istediğimiz için 'ssss ' ve ' eeee' seçebiliriz.
- Başlangıç karakteri olan 'ssss ' ifâdesinin sonunda, bitiş karakteri olan ' eeee' ifâdesinin başında boşluk karakteri olduğuna dikkat ediniz. Bu, bu kelîmelerin diğer kelîmelerle birleşmemesi için şarttır.

- Veri ön işleme işlemi yapıldı. Bir kısım düzeltme elle yapıldı (uygunsuz cümle silme, çeviri düzeltme vs.);
- fakat asıl veri ön işleme işlemi adımları 'VeriOnIsleme.ipynb' dosyasında mevcuttur

In [2]:
# Hiperparametreler:

# 'Modeli eğit' bayrağı:
trainModel = False

# Veri seti hiperparametreleri:
useAllDataSet = False# Tüm veri setini kullan = True
limitCountOfSentences = 400000# Tüm veri setini kullanılmayacaksa, cümle sayısı

# Tokenizer ve Embedding hiperparametreleri:
embeddingSize = 100# Kelîme gömme boyutu
useManuelMaxLengthForSentences = False# Cümle uzunluğunu elle kısaltmak için bu seçeneği 'True' olarak bırakın
# Yukarıdaki ayarı 'False' olarak belirlerseniz, veri setindeki en uzun cümle, azamî cümle uzunluğu olarak seçilir.
maxLengthForTRSentences = 9# Türkçe metînler için (kod çözücü) cümle uzunluğu
maxLengthForENGSentences = 8# Türkçe metînler için (kod çözücü) cümle uzunluğu
loadEmbeddingWeightsFromFiles = True# Ağırlıkları bir kez kaydettikten sonra hız ve bellek kazanımı için bu parametreyi açın
# Eğer içeri aktardığınız cümle sayısı değişirse, yukarıdaki parametreyi 'False' yapın

# Model hiperparametreleri:
gruCellSize = 196# GRU katmanlarının (6 adet) hücre sayısı (İşlem gücünüz varsa, 256 veyâ daha fazla verin)
epochNum = 10# Tur ('epoch') sayısı
batchSizeNum = 90# İşlem gücünüz varsa 96, 128, 256 ve daha fazlasını seçebilirsiniz
learningRate = 0.004# RMSProp optimizasyon algoritması için öğrenme katsayısı (hızı)

# Modeli yeniden eğitirken model ağırlıklarının dosyadan yüklenmesi için bu ayarı True yapın:
loadModelWeights = True# Model ağırlığı 'modelSavePath' isimli değişkendeki dosya yolundaki dosyadan alınır

In [3]:
# Dosya yolları:
trModelPath = r"trmodel"# Türkçe dil modeli
engModelPath = r"glove.6B.100d.txt"# İngilizce dil modeli
dataSetPath = r"cleaned703KSentences.csv"

modelSavePath = 'nmt_v2.weights.h5'# Model ağırlığının kaydedilmesinde ve / veyâ geri yüklenmesinde kullanılır

# Ağırlıkları dosyadan yüklemek için numpy dizilerinin dosya yolu:
additionalPart = ('allSentences' if useAllDataSet else str(limitCountOfSentences) + 'Sentences' + str(embeddingSize) + 'D')
# Yukarıdaki kod, farklı embeddingSize ve cümle sayısı seçildiğinde ağırlıkların farklı dosyalara kaydedilmesi içindir.
weightsOfEmbeddingEncoderPath = 'weightsOfEmbeddingEncoder'+ additionalPart + '.dat'
weightsOfEmbeddingDecoderPath = 'weightsOfEmbeddingDecoder' + additionalPart + '.dat'

In [4]:
# Farklı uzunluktaki cümleler için kelîme vektörleri dil modelinden alınmalı:
if not loadEmbeddingWeightsFromFiles:
    from gensim.models import Word2Vec# Dil modeli
    from gensim.models import KeyedVectors# Word2Vec dil modelini içe aktarmak için

In [5]:
# Veri setinin içeri aktarılması:
sayac = 0
dataEng = []
dataTR = []
with open(dataSetPath, 'r', encoding='utf8') as file:
    for line in file:
        if sayac == 0:
            sayac += 1
            continue
        parts = line.split("\t")
        dataTR.append(parts[0].strip())
        dataEng.append(parts[1].strip())
        if not useAllDataSet:
            if sayac == limitCountOfSentences:
                break
            sayac += 1
print(f"İçeri aktarılan cümle sayısı: {len(dataTR)}")

İçeri aktarılan cümle sayısı: 400000


In [6]:
# Dil modellerinin içe aktarılması:
if not loadEmbeddingWeightsFromFiles:
    modelTR = KeyedVectors.load_word2vec_format(trModelPath, binary = True)
    try:
        modelEng = KeyedVectors.load_word2vec_format(engModelPath + ".word2vec", binary = False)
    except:
        from gensim.scripts.glove2word2vec import glove2word2vec# GLoVe dil modelini Word2Vec formatına çevirmek için
        glove2word2vec(engModelPath, engModelPath + ".word2vec")# İngilizce modeli word2vec formatına çevirir
        #ve kaynak dizinde '.word2vec' uzantısıyla kaydeder.

In [7]:
# Başlama ve bitiş simgelerinin hedef cümlelere (Türkçe cümlelere) eklenmesi:
startToken = "ssss "
endToken = " eeee"
dataTR = [startToken + i + endToken for i in dataTR]
print(dataTR[10])

ssss harika! eeee


- Tokenizer (Simgeleştirici) ile verileri simgeleştirmeliyiz; fakat mevcut Tokenizer'da eksik olan metni ters çevirme ve simge dizisinden metne çevirme gibi özellikleri eklemeliyiz.
- Ayrıca bu işlemden sonra uygulanan `pad_sequences()` yöntemini de buradan çalıştırmak ve hangi yapı için hangi parametreleri verdiğimizi daha kolay görmek için bu işlemi de alt sınıflayarak `Tokenizer` bünyesine katabiliriz.

In [8]:
# TokenizerExtended tanımı:
class TokenizerExtended(Tokenizer):
    def __init__(self, texts, num_words = None, reverse = False, truncating = True, padding = 'pre',
                maxLengthOfSentence = None):
        Tokenizer.__init__(self, num_words = num_words)
        self.fit_on_texts(texts)
        self.sequences = self.texts_to_sequences(texts)
        if reverse:# Metni ters çevir, metnin sonundan kes; başı daha mühim
            texts = [sent[::-1] for sent in texts]
            truncating = 'pre'
        else:
            truncating = 'post'
        if maxLengthOfSentence is None:# Azamî uzunluk belirtilmemişse..
            lens = nm.asarray([len(i) for i in self.sequences])
            self.maxToken = lens.max()
        else:
            self.maxToken = maxLengthOfSentence
        self.sequencesPadded = pad_sequences(self.sequences, maxlen = self.maxToken,
                                             truncating=truncating, padding=padding)

    def tokenToWord(self, token):
        try:
            return ' ' if token == 0 else self.index_word[token]
        except:# Simgeleyicini sözlüğünde olmayan bir simge verildiyse, boşluk karakterini döndür
            return ' '

    def sentenceToSequences(self, sentence, reverse = False, padding = 'post'):
        sentence = sentence
        if reverse:
            sentence = " ".join(sentence.split()[::-1])
            truncating = 'pre'
        else:
            truncating = 'post'
        seq = self.texts_to_sequences([sentence])
        seqPadded = pad_sequences(seq, maxlen = self.maxToken, truncating = truncating, padding = padding)
        return nm.asarray(seqPadded)

In [9]:
# Veri format tamâmlama:
if useManuelMaxLengthForSentences:
    tokenizerEncoder = TokenizerExtended(dataEng, padding='pre', reverse=True, truncating='pre',
                                     maxLengthOfSentence = maxLengthForENGSentences)
    tokenizerDecoder = TokenizerExtended(dataTR, padding='post', reverse=False, truncating='post',
                                     maxLengthOfSentence = maxLengthForTRSentences)
else:
    tokenizerEncoder = TokenizerExtended(dataEng, padding='pre', reverse=True, truncating='pre')
    tokenizerDecoder = TokenizerExtended(dataTR, padding='post', reverse=False, truncating='post')
# Kodlayıcı cümlelerinin ters çevrilmesinin sebebi verimliliktir.
# Metîn uzunluğunu 12 olarak belirledik; çünkü bu, göstermelik bir model ve metînlerimiz gâyet kısa..

In [10]:
# Kısayollar:
sequencesSrc = tokenizerEncoder.sequencesPadded
sequencesTarget = tokenizerDecoder.sequencesPadded

In [11]:
# Dil modellerinin kelîme vektör boyutunu istediğimiz boyuta, anlamı fazla kaybetmeden dönüştürmek için bir fonksiyon:
from math import ceil
def shrinkWordVectors(word2VecModel, targetDimensionLength, sourceTokenizer: Tokenizer, asList = False):
    """
        Bu, eğitilmiş dil modellerinin vektör boyutunu anlamı fazla kaybetmeden küçültmek için bir fonksiyondur.
        Eğer sadece dil modelinden ilgili ağırlıkların liste veyâ sözlük olarak alınması isteniyorsa (boyut korunarak)
        targetDimensionLength parametresine ilgili dil modelinin boyut uzunluğu verilmelidir.
        word2VecModel: 'gensim.models.KeyedVectors' tipinde olmalıdır
        targetDimensionLength değeri verilen modelin vektör boyutundan küçük olmalıdır.
        Eğer kaynağın boyut uzunluğundan daha büyük bir sayı `targetDimensionLength` olarak verilirse `None` döndürülür
        sourceTokenizer: Verilen modeldeki tüm kelîmeler yerine sadece verilen 'Tokenizer' nesnesindeki kelîmelerin
        hedef boyuta uygun vektörü isteniyorsa, bu parametre sağlanmalıdır.
        asList : Eğer bir 'Tokenizer' sağlanırsa ve ağırlıklar Tokenizer'ın kelîmelerinin sırasıyla dizilmiş bir liste olarak
        istenirse, bu parametreye `True` değeri verilmelidir.

        'Tokenizer' 0 indisini kullanmadığından dizi olarak dönen değer len(tokenizer.index_word) + 1 olmaktadır.
    """
    embeddingSize = word2VecModel.vector_size
    if embeddingSize < targetDimensionLength:
        return None
    stepSize = ceil((embeddingSize / targetDimensionLength))
    newWordToIndex = {}
    if sourceTokenizer is not None:
        newEmbeddings = nm.random.uniform(low = -1, high = 1, size=(len(sourceTokenizer.word_index) + 1, targetDimensionLength))
        for word, sayac in sourceTokenizer.word_index.items():
            try:
                vect = word2VecModel.get_vector(word)
                for i in range(0, targetDimensionLength):
                    newEmbeddings[sayac][i] = vect[i*stepSize:(i+1)*stepSize].sum()
                if not asList:
                    newWordToIndex[word] = newEmbeddings[sayac]
            except:
                pass
    else:
        words = modelTR.index_to_key
        newEmbeddings = {}
        if word in words:
            try:
                vect = word2VecModel.get_vector(word)
                newVect = nm.zeros(targetDimensionLength)
                for i in range(0, targetDimensionLength):
                    newVect[i] = vect[i*stepSize:(i+1)*stepSize].sum()
                newEmbeddings[word] = newVect
            except:
                pass
    if (sourceTokenizer is not None) and (not asList):
        return newWordToIndex
    return newEmbeddings

In [12]:
# Eğitilmiş dil modellerinden kelîmeler için katsayıları alma
# (bu katsayılar Embedding'e verilerek, Embedding katmanının kayıp üzerindeki etkisi azaltılıyor):
if not loadEmbeddingWeightsFromFiles:
    weightsOfEmbeddingEncoder = shrinkWordVectors(modelEng, embeddingSize, tokenizerEncoder, asList = True)
    weightsOfEmbeddingDecoder = shrinkWordVectors(modelTR, embeddingSize, tokenizerDecoder, asList = True)
    # Embedding katmanı için ağırlıkları kaydet:
    weightsOfEmbeddingEncoder.tofile(weightsOfEmbeddingEncoderPath)
    weightsOfEmbeddingDecoder.tofile(weightsOfEmbeddingDecoderPath)
else:
    weightsOfEmbeddingEncoder = nm.fromfile(weightsOfEmbeddingEncoderPath, dtype = nm.float64)
    sentNum = weightsOfEmbeddingEncoder.shape[0] // embeddingSize
    weightsOfEmbeddingEncoder = weightsOfEmbeddingEncoder.reshape((sentNum, embeddingSize))

    weightsOfEmbeddingDecoder = nm.fromfile(weightsOfEmbeddingDecoderPath, dtype = nm.float64)
    sentNum = weightsOfEmbeddingDecoder.shape[0] // embeddingSize
    weightsOfEmbeddingDecoder = weightsOfEmbeddingDecoder.reshape((sentNum, embeddingSize))

In [13]:
# Yaptığımız işlem büyük boyutlu hafîza gerektirdiği için, artık işimiz olmayan dil modellerini hâfızadan silebiliriz:
if not loadEmbeddingWeightsFromFiles:
    del modelTR, modelEng

In [14]:
# Encoder katmanlarının oluşturulması:
encoderInput = Input(shape=(None,), name='encoderInput')
encoderEmbedding = Embedding(input_dim=len(tokenizerEncoder.word_index) + 1,
                             output_dim=embeddingSize,
                            weights=[weightsOfEmbeddingEncoder],
                            name = 'encoderEmbedding')

encoderGRU1 = GRU(gruCellSize, return_sequences = True, name='encoderGRU1')
encoderGRU2 = GRU(gruCellSize, return_sequences = True, name='encoderGRU2')
encoderGRU3 = GRU(gruCellSize, return_sequences = False, name='encoderGRU3')

In [15]:
# Encoder katmanlarının bağlanması:
encoder = encoderEmbedding(encoderInput)
encoder = encoderGRU1(encoder)
encoder = encoderGRU2(encoder)
encoder = encoderGRU3(encoder)
encoderOutput = encoder

In [16]:
# Decoder katmanlarının oluşturulması:
decoderInput = Input(shape=(None,), name = 'decoderInput')
decoderEmbedding = Embedding(input_dim = len(tokenizerDecoder.word_index) + 1,
                            output_dim = embeddingSize,
                            weights=[weightsOfEmbeddingDecoder],
                            name = 'decoderEmbedding')
decoderGRU1 = GRU(gruCellSize, return_sequences = True, name='decoderGRU1')
decoderGRU2 = GRU(gruCellSize, return_sequences = True, name='decoderGRU2')
decoderGRU3 = GRU(gruCellSize, return_sequences = True, name='decoderGRU3')
# Çıktımız kodlayıcıdaki gibi düşünce vektörü değil, cümle olmalı; bu sebeple son GRU katmanı da dizi döndürmeli
# Son katmandan çıkan dizinin, hangi elemana denk düştüğünü anlamalıyız.
# Bunun için kelîme sayısı kadar hücre bulunduran bir katman eklemeliyiz:
decoderDense = Dense(len(tokenizerDecoder.word_index) + 1, name = 'decoderDense', activation = 'softmax')
# Hangi sınıfın değeri daha yüksekse o kelîmenin üretilmesi için 'softmax' aktivasyon seçilmeli;

In [17]:
# Decoder katmanlarının bağlanması:
decoder = decoderEmbedding(decoderInput)
decoder = decoderGRU1(decoder, initial_state = encoderOutput)
decoder = decoderGRU2(decoder, initial_state = encoderOutput)
decoder = decoderGRU3(decoder, initial_state = encoderOutput)

decoder = decoderDense(decoder)
decoderOutput = decoder

In [18]:
# Modellerin oluşturulması ve eğitim modelinin derlenmesi:
# Birden fazla model oluşturmalıyız; çünkü eğitim modelinde kodlayıcıya iki girdi veriyoruz,
# çeviri yaparken bir girdi vermeliyiz

modelTrain = Model(inputs=[encoderInput, decoderInput], outputs=[decoderOutput])
modelTrain.compile(optimizer = RMSprop(learning_rate = learningRate),
                   loss = SparseCategoricalCrossentropy(reduction = 'mean'))

modelEncoder = Model(inputs=[encoderInput], outputs=[encoderOutput])
modelDecoder = Model(inputs = [decoderInput, encoderOutput], outputs = [decoderOutput])
# Modeli ayrı tanımladığımızda, ikinci girdiyi de 'inputs' kısmında belirtmeliyiz.
modelTrain.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoderInput (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoderEmbedding (Embedding)  │ (None, None, 100)         │       2,492,800 │ encoderInput[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoderGRU1 (GRU)             │ (None, None, 196)         │         175,224 │ encoderEmbedding[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoderInput (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoderGRU2 (GRU)             │ (None, None, 196)         │         231,672 │ encoderGRU1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoderEmbedding (Embedding)  │ (None, None, 100)         │       8,183,700 │ decoderInput[1][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ encoderGRU3 (GRU)             │ (None, 196)               │         231,672 │ encoderGRU2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoderGRU1 (GRU)             │ (None, None, 196)         │         175,224 │ decoderEmbedding[0][0],    │
│                               │                           │                 │ encoderGRU3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoderGRU2 (GRU)             │ (None, None, 196)         │         231,672 │ decoderGRU1[0][0],         │
│                               │                           │                 │ encoderGRU3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoderGRU3 (GRU)             │ (None, None, 196)         │         231,672 │ decoderGRU2[0][0],         │
│                               │                           │                 │ encoderGRU3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoderDense (Dense)          │ (None, None, 81837)       │      16,121,889 │ decoderGRU3[0][0]          │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 28,075,525 (107.10 MB)

 Trainable params: 28,075,525 (107.10 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Modeli kaydetmek için 'kayıt için model çağrısı' oluşturmalıyız:
saveCheckpoint = ModelCheckpoint(modelSavePath, save_weights_only = True)
# Bu yolda bir dosya varsa, o dosyanın üzerine yazılır

In [26]:
# Model ağırlıklarını dosyadan yükleme (eğer bu ayar açıksa):
if loadModelWeights:
    modelTrain.load_weights(modelSavePath)

In [21]:
# Girdi verilerinin tanımlanması:
xData = {'encoderInput' : sequencesSrc, 'decoderInput' : sequencesTarget[:,:-1]}
yData = sequencesTarget[:,1:]

# Kodlayıcı girdi verisi İngilizce cümlenin dizisel hâlidir (Tokenizer + pad_sequences'tan geçmiş)
# !!! Kod çözücü girdi verisi son kelîme hâriç, önceki kelîmelerden oluşan dizidir;
# !!! Kod çözücü çıktı verisi ilk kelîme hâriç sonraki tüm kelîmelerden oluşan dizidir
# Kod çözücü, her adımda sonraki kelîmeyi tahmîn etme üzerine eğitilen modeldir.

In [22]:
# Model eğitimi:
if trainModel:
    epochNum = 6
    modelTrain.fit(x = xData, y = yData,
                   epochs = epochNum,
                   callbacks = [saveCheckpoint],
                   batch_size = batchSizeNum)

In [23]:
# Çeviri fonksiyonu:
def translate(sentence):
    sequences = tokenizerEncoder.sentenceToSequences(sentence, padding = 'pre', reverse = True)# Cümle -> simge dizisi
    thoughtMatrix = modelEncoder.predict(sequences)# Düşünce vektörü
    maxTokens = tokenizerDecoder.maxToken# Azamî cümle uzunluğu
    decoderInput = nm.zeros(shape = (1, maxTokens), dtype = nm.int32)
    decoderInput[0][0] = tokenizerDecoder.word_index[startToken.strip()]# Girdinin ilk elemanı başlangıç simgesi olmalı
    xData = [decoderInput, thoughtMatrix]# Buradaki sıra, model tanımlanırken verilen sırayla aynı olmalı
    result = ''# Üretilen cümle
    sayac = 0
    while sayac < maxTokens:# Üretim en fazla azamî cümle uzunluğuna kadar devâm etmektedir
        vect = modelDecoder.predict(xData)# Model tahmini, kelîme sayısı uzunluğunda bir tek nokta vektörüdür
        releatedOneHot = vect[0,sayac,:]# Kelîme için üretilen tek nokta vektörü
        token = nm.argmax(releatedOneHot)# Tek nokta vektöründe, değeri 1 olan elemanın indisi, üretilen simgedir.
        if token == tokenizerDecoder.word_index[endToken.strip()]:# Bitiş simgesi üretildiyse, kelîme üretmeyi bırak
            break
        result += ' ' + tokenizerDecoder.tokenToWord(token)
        sayac += 1
        if sayac != 0 and sayac < maxTokens:
            decoderInput[0][sayac] = token# ÖNEMLİ : Sonraki kelîme üretilirken, bu üretilen kelîmeyi girdi olarak vermeliyiz
    return result

In [27]:
text = "this is a pencil"# Modeli iyi eğittiyseniz, daha uzun ve karışık bir cümle deneyebilirsiniz
print(f"Kaynak metîn : {text}")
print(f"Çeviri sonucu : {translate(text)}")

Kaynak metîn : this is a pencil
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Çeviri sonucu :  bu bir kalem


In [28]:
text = "who are you?"# Modeli iyi eğittiyseniz, daha uzun ve karışık bir cümle deneyebilirsiniz
print(f"Kaynak metîn : {text}")
print(f"Çeviri sonucu : {translate(text)}")

Kaynak metîn : who are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Çeviri sonucu :  sen kimsin


- Eğer modeli 400 bin kelîme, 100 kelîme vektörü ve 200+ GRU hücre sayısı ile 10-15 epoch eğitirseniz 5 - 6 kelîmelik yaygın cümleleriniz doğru şekilde çevrilebilir; fakat veri seti kısa cümlelerden oluştuğundan cümleniz isim cümlesi (noun clause), sıfat cümlesi (noun clause) gibi karmaşık yapıları, sık kullanılmayan kelîmeleri çeviremeyecektir.